In [1]:
import numpy as np
import pandas as pd

In [2]:
# ! pip install transformers datasets
# ! pip install adapter-transformers
# ! pip install scipy sklearn
# ! pip install evaluate

In [3]:
import json
import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
df = pd.read_json(r'data\citation.jsonl', lines=True)
df = df.drop('metadata', axis=1)

In [5]:
df.head()

,text,label
0,"Thus , over the past few years , along with ad...",Background
1,"This was done by MERT optimization ( Och , 200...",Uses
2,"She evaluates 3,000 German verbs with a token ...",Background
3,The following four components have been identi...,Background
4,Briscoe and Carroll ( 1997 ) report on manuall...,CompareOrContrast


In [6]:
labels_df = df.label.unique()
labels_df

array(['Background', 'Uses', 'CompareOrContrast', 'Extends', 'Motivation',
       'Future'], dtype=object)

In [7]:
from datasets import load_dataset
dataset = load_dataset("json", data_files={"train": "data\\citation.jsonl", "test": "data\\citation_test.jsonl"})
dataset = dataset.remove_columns(["metadata"])
dataset["train"].features

Using custom data configuration default-db179d5a34c6ea0d
Reusing dataset json (C:\Users\Sudeshna_Dash\.cache\huggingface\datasets\json\default-db179d5a34c6ea0d\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


  0%|          | 0/2 [00:00<?, ?it/s]

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None)}

In [8]:
dataset["test"]["label"]

['CompareOrContrast',
 'Background',
 'Motivation',
 'Background',
 'Uses',
 'Background',
 'Background',
 'Background',
 'Background',
 'Background',
 'Background',
 'CompareOrContrast',
 'CompareOrContrast',
 'CompareOrContrast',
 'Background',
 'Motivation',
 'Background',
 'Uses',
 'Uses',
 'Background',
 'CompareOrContrast',
 'Future',
 'Background',
 'CompareOrContrast',
 'CompareOrContrast',
 'Background',
 'Background',
 'Uses',
 'Uses',
 'CompareOrContrast',
 'Background',
 'Background',
 'Background',
 'Uses',
 'Uses',
 'Uses',
 'Background',
 'Uses',
 'CompareOrContrast',
 'Background',
 'Background',
 'Uses',
 'Background',
 'Background',
 'CompareOrContrast',
 'CompareOrContrast',
 'CompareOrContrast',
 'Background',
 'CompareOrContrast',
 'Background',
 'CompareOrContrast',
 'Background',
 'Uses',
 'CompareOrContrast',
 'CompareOrContrast',
 'Background',
 'Background',
 'CompareOrContrast',
 'Uses',
 'CompareOrContrast',
 'Background',
 'Background',
 'Uses',
 'Backgroun

In [9]:
dataset['train'] = dataset['train'].class_encode_column("label")

Loading cached processed dataset at C:\Users\Sudeshna_Dash\.cache\huggingface\datasets\json\default-db179d5a34c6ea0d\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253\cache-f71c04cc55d20d46.arrow
Loading cached processed dataset at C:\Users\Sudeshna_Dash\.cache\huggingface\datasets\json\default-db179d5a34c6ea0d\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253\cache-7f605b2416bc16c3.arrow


In [10]:
dataset['test'] = dataset['test'].class_encode_column("label")

Loading cached processed dataset at C:\Users\Sudeshna_Dash\.cache\huggingface\datasets\json\default-db179d5a34c6ea0d\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253\cache-569a3b733759075b.arrow
Loading cached processed dataset at C:\Users\Sudeshna_Dash\.cache\huggingface\datasets\json\default-db179d5a34c6ea0d\0.0.0\a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253\cache-f65684af50a127c4.arrow


In [11]:
dataset['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=6, names=['Background', 'CompareOrContrast', 'Extends', 'Future', 'Motivation', 'Uses'], id=None)}

In [12]:
id2labeldict = {}
for i in range(0, labels_df.size):
    id2labeldict[i] = labels_df[i]

In [17]:
tokenized_citation = dataset.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
tokenized_citation["train"][0]

{'text': 'Thus , over the past few years , along with advances in the use of learning and statistical methods for acquisition of full parsers ( Collins , 1997 ; Charniak , 1997a ; Charniak , 1997b ; Ratnaparkhi , 1997 ) , significant progress has been made on the use of statistical learning methods to recognize shallow parsing patterns syntactic phrases or words that participate in a syntactic relationship ( Church , 1988 ; Ramshaw and Marcus , 1995 ; Argamon et al. , 1998 ; Cardie and Pierce , 1998 ; Munoz et al. , 1999 ; Punyakanok and Roth , 2001 ; Buchholz et al. , 1999 ; Tjong Kim Sang and Buchholz , 2000 ) .',
 'label': 0,
 'input_ids': [0,
  42702,
  2156,
  81,
  5,
  375,
  367,
  107,
  2156,
  552,
  19,
  9766,
  11,
  5,
  304,
  9,
  2239,
  8,
  17325,
  6448,
  13,
  3857,
  9,
  455,
  28564,
  268,
  36,
  5415,
  2156,
  7528,
  25606,
  732,
  4422,
  20082,
  2156,
  7528,
  102,
  25606,
  732,
  4422,
  20082,
  2156,
  7528,
  428,
  25606,
  12041,
  282,
  111

#  Adapter based Training


In [ ]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

dataset = dataset.map(encode_batch, batched=True)


In [ ]:
# The transformers model expects the target class column to be named "labels"
dataset = dataset.rename_column("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=labels_df.size,
)

In [ ]:
from transformers import RobertaAdapterModel

model = RobertaAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

In [ ]:
# Add a new adapter
model.add_adapter("roberta_base_rct")
model.add_classification_head(
    "roberta_base_rct",
    num_labels=labels_df.size,
    id2label=id2labeldict
  )
# Activate the adapter
model.train_adapter("roberta_base_rct")

In [ ]:
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./jupyter_rct_adapter",
    overwrite_output_dir=True,
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_accuracy,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()